In [1]:
!ls -l dataset/

total 174000
drwxr-xr-x 2 root root      4096 Jun 14 14:09 1
-rw-r--r-- 1 root root 178170641 Jun 20 05:53 conn.log


In [1]:
import json
from tqdm import tqdm
ll = []
def read_zeek_files(filename):
    file_time = {}
    cache_lines = {}
    lines = 0
    if not filename.endswith('.log'):
        filename += '.log'
    filename_without_ext = filename.split('/')[-1].split('.')[0]
    file_handler = open(filename, 'r')
#     pbar = tqdm(total = 60000000)
    while True:
#         tqdm.monitor()
        zeek_line = file_handler.readline()
        if not zeek_line:
            return
        try:
            nline = json.loads(zeek_line)
            line = {'type': filename, 'data': nline}
            timestamp = nline.get('ts', 0)
#             print("json works")
        except json.decoder.JSONDecodeError:
            nline = zeek_line
            if not nline or nline[0] == '#':
                print("nline comment or over")
                continue
            line = {'type': filename, 'data': nline}
            timestamp = nline.split('\t')[0]
            print("json decode error")
        try:
            file_time[filename] = float(timestamp)
            cache_lines[filename] = line
        except ValueError:
            pass
        files_sorted_by_ts = sorted(file_time, key=file_time.get)
        try:
            file_with_earliest_flow = files_sorted_by_ts[0]
        except IndexError:
            print("index error")
        line_to_send = cache_lines[file_with_earliest_flow]
#         print('	> Sent Line: {}'.format(line_to_send), 0, 3)
#         ll.append(line_to_send)
        del cache_lines[file_with_earliest_flow]
        del file_time[file_with_earliest_flow]
        lines += 1
        yield line_to_send
#         pbar.update(1)
#     file_handler.close()
#     return lines


In [2]:
import json
from tqdm import tqdm
ll = []
def read_zeek_files2(filename):
    file_time = {}
    cache_lines = {}
    lines = 0
    if not filename.endswith('.log'):
        filename += '.log'
    filename_without_ext = filename.split('/')[-1].split('.')[0]
    file_handler = open(filename, 'r')
    pbar = tqdm(total = 6000)
    while lines < 6000:
#         tqdm.monitor()
        zeek_line = file_handler.readline()
        if not zeek_line:
            return
        try:
            nline = json.loads(zeek_line)
            line = {'type': filename, 'data': nline}
            timestamp = nline.get('ts', 0)
#             print("json works")
        except json.decoder.JSONDecodeError:
            nline = zeek_line
            if not nline or nline[0] == '#':
                print("nline comment or over")
                continue
            line = {'type': filename, 'data': nline}
            timestamp = nline.split('\t')[0]
            print("json decode error")
        try:
            file_time[filename] = float(timestamp)
            cache_lines[filename] = line
        except ValueError:
            pass
        files_sorted_by_ts = sorted(file_time, key=file_time.get)
        try:
            file_with_earliest_flow = files_sorted_by_ts[0]
        except IndexError:
            print("index error")
        line_to_send = cache_lines[file_with_earliest_flow]
#         print('	> Sent Line: {}'.format(line_to_send), 0, 3)
        ll.append(line_to_send)
        del cache_lines[file_with_earliest_flow]
        del file_time[file_with_earliest_flow]
        lines += 1
#         yield line_to_send
        pbar.update(1)
    file_handler.close()
    return lines


In [3]:
j = 0
for i in read_zeek_files("dataset/1/conn.log"):
    print(i)
    j+=1
    if (j == 5):
        break
        
    

{'type': 'dataset/1/conn.log', 'data': {'ts': 1331901000.0, 'uid': 'CUySP6353GMmKYJETb', 'id.orig_h': '192.168.202.79', 'id.orig_p': 46117, 'id.resp_h': '192.168.229.254', 'id.resp_p': 443, 'proto': 'tcp', 'conn_state': 'SF', 'missed_bytes': 0, 'history': '^dDafFr', 'orig_pkts': 3, 'orig_ip_bytes': 382, 'resp_pkts': 9, 'resp_ip_bytes': 994, 'vlan': 120, 'orig_l2_addr': '00:0c:29:41:4b:e7', 'resp_l2_addr': '00:16:47:9d:f2:c2', 'emoji': ''}}
{'type': 'dataset/1/conn.log', 'data': {'ts': 1331901000.0, 'uid': 'CctNbQQy6OKXKEHmg', 'id.orig_h': '192.168.202.79', 'id.orig_p': 50463, 'id.resp_h': '192.168.229.251', 'id.resp_p': 80, 'proto': 'tcp', 'conn_state': 'SH', 'missed_bytes': 0, 'history': 'Fa', 'orig_pkts': 1, 'orig_ip_bytes': 52, 'resp_pkts': 1, 'resp_ip_bytes': 52, 'vlan': 120, 'orig_l2_addr': '00:0c:29:41:4b:e7', 'resp_l2_addr': '00:16:47:9d:f2:c2', 'emoji': ''}}
{'type': 'dataset/1/conn.log', 'data': {'ts': 1331901000.0, 'uid': 'C4CRnB4oVj0mx9Vsjg', 'id.orig_h': '192.168.202.79', '

In [4]:
# read_zeek_files2("dataset/1/conn.log")

In [5]:
# for i in ll[0:10]:
#     print(i["data"]["uid"])

In [6]:
# from StratosphereLinuxIPS.slips_files.common.slips_utils import utils

In [7]:
from database import __database__

In [8]:
port = 6379

In [9]:
__database__.connect_to_redis_server(port)

True

In [10]:
from tzlocal import get_localzone
local_timezone = get_localzone()

In [11]:
from datetime import datetime,timedelta

def get_time(time: str) -> datetime:
    timeformat=None
    """
    Take time in string and return datetime object.
    The format of time can be completely different. It can be seconds, or dates with specific formats.
    If user does not define the time format in configuration file, we have to try most frequent cases of time formats.
    :param time: epoch time
    """
    
    if not timeformat:
        # The time format was not defined from configuration file neither from last flows.
        timeformat = utils.define_time_format(time)
    if not timeformat:
        # We did not find the right time format.
        outputqueue.put(
            '01|profiler|[Profile] We did not find right time format. Please set the time format in the configuration file.'
        )
    defined_datetime: datetime = None
    if timeformat:
        if timeformat == 'unixtimestamp':
            # The format of time is in epoch unix timestamp.
            # Correct datetime according to the current timezone
            defined_datetime = datetime.fromtimestamp(
                float(time), local_timezone
            )
        else:
            try:
                # The format of time is a complete date.
                # Dont modify it, since
                # 1) The time is a string, so we dont know the original timezone
                # 2) the python call datetime.fromtimestamp uses by default
                # the local zone when nothing is specified.
                # https://docs.python.org/3/library/datetime.html#datetime.timezone
                # convert epoch to datetime obj and use the current timezone
                # self.print(time)
                # self.print(self.local_timezone)
                # defined_datetime = datetime.strptime(time, self.timeformat)#.astimezone(self.local_timezone)
                # defined_datetime = datetime.fromtimestamp(float(time), self.local_timezone)
                # defined_datetime = datetime.fromtimestamp(float(time), self.local_timezone)
                # convert dt obj to user specified tiemformat
                # defined_datetime = defined_datetime.strftime(self.timeformat)
                defined_datetime = time
            except ValueError:
                defined_datetime = None
    else:
        # We do not know the time format so we can not read it.
        outputqueue.put(
            '01|profiler|[Profile] We did not find right time format. Please set the time format in the configuration file.'
        )

    # if defined_datetime is None and self.timeformat:
    # There is suricata issue with invalid timestamp for examaple: "1900-01-00T00:00:08.511802+0000"
    # pass
    return defined_datetime


In [12]:

def process_line(new_line):
    column_values = {}
    line = new_line['data']
    file_type = new_line['type']
    # all zeek lines recieved from stdin should be of type conn
    if file_type == 'stdin' and new_line.get('line_type', False) == 'zeek':
        file_type = 'conn'
    else:
        # if the zeek dir given to slips has 'conn' in it's name,
        # slips thinks it's reading a conn file
        # because we use the file path as the file 'type'
        # to fix this, only use the file name as file 'type'
        file_type = file_type.split('/')[-1]
    
    column_values['type'] = ''
    ts = line.get('ts', False)
    if ts:
        column_values['starttime'] = get_time(ts)
    else:
        column_values['starttime'] = ''

    column_values['uid'] = line.get('uid', False)
    column_values['saddr'] = line.get('id.orig_h', '')
    column_values['daddr'] = line.get('id.resp_h', '')
    
    if 'conn' in file_type:
            # {'ts': 1538080852.403669, 'uid': 'Cewh6D2USNVtfcLxZe', 'id.orig_h': '192.168.2.12', 'id.orig_p': 56343,
            # 'id.resp_h': '192.168.2.1', 'id.resp_p': 53, 'proto': 'udp', 'service': 'dns', 'duration': 0.008364,
            # 'orig_bytes': 30, 'resp_bytes': 94, 'conn_state': 'SF', 'missed_bytes': 0, 'history': 'Dd', 'orig_pkts': 1,
            # 'orig_ip_bytes': 58, 'resp_pkts': 1, 'resp_ip_bytes': 122, 'orig_l2_addr': 'b8:27:eb:6a:47:b8',
            # 'resp_l2_addr': 'a6:d1:8c:1f:ce:64', 'type': './zeek_files/conn'}

            column_values.update(
                {
                    'type': 'conn',
                    'dur': float(line.get('duration', 0)),
                    'endtime': str(column_values['starttime'])
                    + str(timedelta(seconds=float(line.get('duration', 0)))),
                    'proto': line['proto'],
                    'appproto': line.get('service', ''),
                    'sport': line.get('id.orig_p', ''),
                    'dport': line.get('id.resp_p', ''),
                    'state': line.get('conn_state', ''),
                    'dir': '->',
                    'spkts': line.get('orig_pkts', 0),
                    'dpkts': line.get('resp_pkts', 0),
                    'sbytes': line.get('orig_bytes', 0),
                    'dbytes': line.get('resp_bytes', 0),
                    'pkts': line.get('orig_bytes', 0)
                    + line.get('resp_pkts', 0),
                    'bytes': line.get('orig_bytes', 0)
                    + line.get('resp_bytes', 0),
                    'state_hist': line.get(
                        'history', line.get('conn_state', '')
                    ),
                    'smac': line.get('orig_l2_addr', ''),
                    'dmac': line.get('resp_l2_addr', ''),
                }
            )
    elif 'dns' in file_type:
        column_values.update(
            {
                'type': 'dns',
                'query': line.get('query', ''),
                'qclass_name': line.get('qclass_name', ''),
                'qtype_name': line.get('qtype_name', ''),
                'rcode_name': line.get('rcode_name', ''),
                'answers': line.get('answers', ''),
                'TTLs': line.get('TTLs', ''),
            }
        )

        if type(column_values['answers']) == str:
            # If the answer is only 1, Zeek gives a string
            # so convert to a list
            column_values.update(
                {'answers': [column_values['answers']]}
            )

    elif 'http' in file_type:
        # {"ts":158.957403,"uid":"CnNLbE2dyfy5KyqEhh","id.orig_h":"10.0.2.105","id.orig_p":49158,"id.resp_h":"64.182.208.181","id.resp_p":80,"trans_depth":1,"method":"GET","host":"icanhazip.com","uri":"/","version":"1.1","user_agent":"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.38 (KHTML, like Gecko) Chrome/45.0.2456.99 Safari/537.38","request_body_len":0,"response_body_len":13,"status_code":200,"status_msg":"OK","tags":[],"resp_fuids":["FwraVxIOACcjkaGi3"],"resp_mime_types":["text/plain"]}
        column_values.update(
            {
                'type': 'http',
                'method': line.get('method', ''),
                'host': line.get('host', ''),
                'uri': line.get('uri', ''),
                'httpversion': line.get('version', 0),
                'user_agent': line.get('user_agent', ''),
                'request_body_len': line.get('request_body_len', 0),
                'response_body_len': line.get('response_body_len', 0),
                'status_code': line.get('status_code', ''),
                'status_msg': line.get('status_msg', ''),
                'resp_mime_types': line.get('resp_mime_types', ''),
                'resp_fuids': line.get('resp_fuids', ''),
            }
        )

    elif 'ssl' in file_type:
        # {"ts":12087.045499,"uid":"CdoFDp4iW79I5ZmsT7","id.orig_h":"10.0.2.105","id.orig_p":49704,"id.resp_h":"195.211.240.166","id.resp_p":443,"version":"SSLv3","cipher":"TLS_RSA_WITH_RC4_128_SHA","resumed":false,"established":true,"cert_chain_fuids":["FhGp1L3yZXuURiPqq7"],"client_cert_chain_fuids":[],"subject":"OU=DAHUATECH,O=DAHUA,L=HANGZHOU,ST=ZHEJIANG,C=CN,CN=192.168.1.108","issuer":"O=DahuaTech,L=HangZhou,ST=ZheJiang,C=CN,CN=Product Root CA","validation_status":"unable to get local issuer certificate"}
        # {"ts":1382354909.915615,"uid":"C7W6ZA4vI8FxJ9J0bh","id.orig_h":"147.32.83.53","id.orig_p":36567,"id.resp_h":"195.113.214.241","id.resp_p":443,"version":"TLSv12","cipher":"TLS_ECDHE_ECDSA_WITH_RC4_128_SHA","curve":"secp256r1","server_name":"id.google.com.ar","resumed":false,"established":true,"cert_chain_fuids":["FnomJz1vghKIOHtytf","FSvQff1KsaDkRtKXo4","Fif2PF48bytqq6xMDb"],"client_cert_chain_fuids":[],"subject":"CN=*.google.com,O=Google Inc,L=Mountain View,ST=California,C=US","issuer":"CN=Google Internet Authority G2,O=Google Inc,C=US","validation_status":"ok"}
        column_values.update(
            {
                'type': 'ssl',
                'sslversion': line.get('version', ''),
                'sport': line.get('id.orig_p', ','),
                'dport': line.get('id.resp_p', ','),
                'cipher': line.get('cipher', ''),
                'resumed': line.get('resumed', ''),
                'established': line.get('established', ''),
                'cert_chain_fuids': line.get('cert_chain_fuids', ''),
                'client_cert_chain_fuids': line.get(
                    'client_cert_chain_fuids', ''
                ),
                'subject': line.get('subject', ''),
                'issuer': line.get('issuer', ''),
                'validation_status': line.get('validation_status', ''),
                'curve': line.get('curve', ''),
                'server_name': line.get('server_name', ''),
                'ja3': line.get('ja3', ''),
                'is_DoH': line.get('is_DoH', 'false'),
                'ja3s': line.get('ja3s', ''),
            }
        )

    elif 'ssh' in file_type:
        column_values.update(
            {
                'type': 'ssh',
                'version': line.get('version', ''),
                'auth_success': line.get('auth_success', ''),
                'auth_attempts': line.get('auth_attempts', ''),
                'client': line.get('client', ''),
                'server': line.get('server', ''),
                'cipher_alg': line.get('cipher_alg', ''),
                'mac_alg': line.get('mac_alg', ''),
                'compression_alg': line.get('compression_alg', ''),
                'kex_alg': line.get('kex_alg', ''),
                'host_key_alg': line.get('host_key_alg', ''),
                'host_key': line.get('host_key', ''),
            }
        )

    elif 'irc' in file_type:
        column_values.update({'type': 'irc'})
    elif 'long' in file_type:
        column_values.update({'type': 'long'})
    elif 'dhcp' in file_type:
        column_values.update(
            {
                'type': 'dhcp',
                'client_addr': line.get('client_addr', ''),
                'server_addr': line.get('server_addr', ''),
                'host_name': line.get('host_name', ''),
                'mac': line.get('mac', ''),  # this is the client mac
                'saddr': line.get('client_addr', ''),
                'daddr': line.get('server_addr', ''),
            }
        )

        # self.column_values['domain'] = line.get('domain','')
        # self.column_values['assigned_addr'] = line.get('assigned_addr','')

        # Some zeek flow don't have saddr or daddr, seen in dhcp.log and notice.log use the mac address instead
        if (
            column_values['saddr'] == ''
            and column_values['daddr'] == ''
            and line.get('mac', False)
        ):
            column_values.update({'saddr': line.get('mac', '')})

    elif 'dce_rpc' in file_type:
        column_values.update({'type': 'dce_rpc'})
    elif 'dnp3' in file_type:
        column_values.update({'type': 'dnp3'})
    elif 'ftp' in file_type:
        column_values.update(
            {
                'type': 'ftp',
                'used_port': line.get('data_channel.resp_p', False),
            }
        )

    elif 'kerberos' in file_type:
        column_values.update({'type': 'kerberos'})
    elif 'mysql' in file_type:
        column_values.update({'type': 'mysql'})
    elif 'modbus' in file_type:
        column_values.update({'type': 'modbus'})
    elif 'ntlm' in file_type:
        column_values.update({'type': 'ntlm'})
    elif 'rdp' in file_type:
        column_values.update({'type': 'rdp'})
    elif 'sip' in file_type:
        column_values.update({'type': 'sip'})
    elif 'smb_cmd' in file_type:
        column_values.update({'type': 'smb_cmd'})
    elif 'smb_files' in file_type:
        column_values.update({'type': 'smb_files'})
    elif 'smb_mapping' in file_type:
        column_values.update({'type': 'smb_mapping'})
    elif 'smtp' in file_type:
        column_values.update(
            {'type': 'smtp', 'last_reply': line.get('last_reply', '')}
        )
    elif 'socks' in file_type:
        column_values.update({'type': 'socks'})
    elif 'syslog' in file_type:
        column_values.update({'type': 'syslog'})
    elif 'tunnel' in file_type:
        column_values.update({'type': 'tunnel'})
    elif 'notice' in file_type:
        """Parse the fields we're interested in in the notice.log file"""
        # notice fields: ts - uid id.orig_h(saddr) - id.orig_p(sport) - id.resp_h(daddr) - id.resp_p(dport) - note - msg
        column_values.update(
            {
                'type': 'notice',
                'sport': line.get('id.orig_p', ''),
                'dport': line.get('id.resp_p', ''),
                # self.column_values['scanned_ip'] = line.get('dst', '')
                'note': line.get('note', ''),
                'msg': line.get(
                    'msg', ''
                ),  # we,'re looking for self signed certs in this field
                'scanned_port': line.get('p', ''),
                'scanning_ip': line.get('src', ''),
            }
        )

        # portscan notices don't have id.orig_h or id.resp_h fields, instead they have src and dst
        if column_values['saddr'] == '':
            column_values.update({'saddr': line.get('src', '')})
        if column_values['daddr'] == '':
            # set daddr to src for now because the notice that contains portscan doesn't have a dst field and slips needs it to work
            column_values.update(
                {'daddr': line.get('dst', column_values['saddr'])}
            )

    elif 'files.log' in file_type:
        """Parse the fields we're interested in in the files.log file"""
        # the slash before files to distinguish between 'files' in the dir name and file.log
        column_values.update(
            {
                'type': 'files',
                'uid': line.get('conn_uids', [''])[0],
                'saddr': line.get('tx_hosts', [''])[0],
                'daddr': line.get('rx_hosts', [''])[0],
                'size': line.get('seen_bytes', ''),  # downloaded file size
                'md5': line.get('md5', ''),
                # used for detecting ssl certs
                'source': line.get('source', ''),
                'analyzers': line.get('analyzers', ''),
                'sha1': line.get('sha1', ''),
            }
        )

    elif 'arp' in file_type:
        column_values.update(
            {
                'type': 'arp',
                'src_mac': line.get('src_mac', ''),
                'dst_mac': line.get('dst_mac', ''),
                'saddr': line.get('orig_h', ''),
                'daddr': line.get('resp_h', ''),
                'dst_hw': line.get('resp_hw', ''),
                'src_hw': line.get('orig_hw', ''),
                'operation': line.get('operation', ''),
            }
        )
    elif 'known_services' in file_type:
        column_values.update(
            {
                'type': 'known_services',
                'saddr': line.get('host', ''),
                # this file doesn't have a daddr field, but we need it in add_flow_to_profile
                'daddr': '0.0.0.0',
                'port_num': line.get('port_num', ''),
                'port_proto': line.get('port_proto', ''),
                'service': line.get('service', ''),
            }
        )
    elif 'software' in file_type:
        software_type = line.get('software_type', '')
        # store info about everything except http:broswer
        # we're already reading browser UA from http.log
        if software_type == 'HTTP::BROWSER':
            return column_values
        column_values.update(
            {
                'type': 'software',
                'saddr': line.get('host', ''),
                'software_type': software_type,
                'unparsed_version': line.get('unparsed_version', ''),
                'version.major': line.get('version.major', ''),
                'version.minor': line.get('version.minor', ''),
            }
        )
    else:
        return False
    return column_values

In [13]:

import sys
import configparser
# from .database import __database__
from StratosphereLinuxIPS.slips_files.common.slips_utils import utils
import ipaddress
import traceback
import requests
import os
import binascii
import base64
import validators
from re import split
from tzlocal import get_localzone
# from .whitelist import Whitelist

In [14]:
# ll[0]

In [15]:
# process_line(ll[0])

In [16]:
# column_values

In [17]:
# ll[0]

In [18]:
# hh = []
# for i in tqdm(ll):
#     l = process_line(i)
#     if l:
#         hh.append(l)

In [19]:
def compute_symbol(
    profileid,
    twid,
    tupleid,
    current_time,
    current_duration,
    current_size,
    tuple_key: str,
):
    """
    This function computes the new symbol for the tuple according to the
    original stratosphere ips model of letters
    Here we do not apply any detection model, we just create the letters
    as one more feature current_time is the starttime of the flow
    """
    try:
        current_duration = float(current_duration)
        current_size = int(current_size)
        now_ts = float(current_time)
#         print(
#             'Starting compute symbol. Profileid: {}, Tupleid {}, time:{} ({}), dur:{}, size:{}'.format(
#                 profileid,
#                 tupleid,
#                 current_time,
#                 type(current_time),
#                 current_duration,
#                 current_size,
#             ),
#             3,
#             0,
#         )
        # Variables for computing the symbol of each tuple
        T2 = False
        TD = False
        # Thresholds learnt from Stratosphere ips first version
        # Timeout time, after 1hs
        tto = timedelta(seconds=3600)
        tt1 = float(1.05)
        tt2 = float(1.3)
        tt3 = float(5)
        td1 = float(0.1)
        td2 = float(10)
        ts1 = float(250)
        ts2 = float(1100)

        # Get the time of the last flow in this tuple, and the last last
        # Implicitely this is converting what we stored as 'now' into 'last_ts' and what we stored as 'last_ts' as 'last_last_ts'
        (last_last_ts, last_ts) = __database__.getT2ForProfileTW(
            profileid, twid, tupleid, tuple_key
        )
        # print(f'Profileid: {profileid}. Data extracted from DB. last_ts: {last_ts}, last_last_ts: {last_last_ts}', 0, 5)

        def compute_periodicity(
            now_ts: float, last_ts: float, last_last_ts: float
        ):
            """Function to compute the periodicity"""
            zeros = ''
            if last_last_ts is False or last_ts is False:
                TD = -1
                T1 = None
                T2 = None
            else:
                # Time diff between the past flow and the past-past flow.
                T1 = last_ts - last_last_ts
                # Time diff between the current flow and the past flow.
                # We already computed this before, but we can do it here again just in case
                T2 = now_ts - last_ts

                # We have a time out of 1hs. After that, put 1 number 0 for each hs
                # It should not happen that we also check T1... right?
                if T2 >= tto.total_seconds():
                    t2_in_hours = T2 / tto.total_seconds()
                    # Shoud round it. Because we need the time to pass to really count it
                    # For example:
                    # 7100 / 3600 =~ 1.972  ->  int(1.972) = 1
                    for i in range(int(t2_in_hours)):
                        # Add the zeros to the symbol object
                        zeros += '0'

                # Compute TD
                try:
                    if T2 >= T1:
                        TD = T2 / T1
                    else:
                        TD = T1 / T2
                except ZeroDivisionError:
                    TD = 1

                # Decide the periodic based on TD and the thresholds
                if TD <= tt1:
                    # Strongly periodicity
                    TD = 1
                elif TD <= tt2:
                    # Weakly periodicity
                    TD = 2
                elif TD <= tt3:
                    # Weakly not periodicity
                    TD = 3
                elif TD > tt3:
                    # Strongly not periodicity
                    TD = 4
#             print(
#                 'Compute Periodicity: Profileid: {}, Tuple: {}, T1={}, T2={}, TD={}'.format(
#                     profileid, tupleid, T1, T2, TD
#                 ),
#                 3,
#                 0,
#             )
            return TD, zeros

        def compute_duration():
            """Function to compute letter of the duration"""
            if current_duration <= td1:
                return 1
            elif current_duration > td1 and current_duration <= td2:
                return 2
            elif current_duration > td2:
                return 3

        def compute_size():
            """Function to compute letter of the size"""
            if current_size <= ts1:
                return 1
            elif current_size > ts1 and current_size <= ts2:
                return 2
            elif current_size > ts2:
                return 3

        def compute_letter():
            """Function to compute letter"""
            # format of this map is as follows
            # {periodicity: {'size' : {duration: letter, duration: letter, etc.}}
            periodicity_map = {
                # every key in this dict represents a periodicity
                '-1': {
                    # every key in this dict is a size 1,2,3
                    # 'size' : {duration: letter, diration: letter, etc.}
                    '1': {'1': '1', '2': '2', '3': '3'},
                    '2': {'1': '4', '2': '5', '3': '6'},
                    '3': {'1': '7', '2': '8', '3': '9'},
                },
                '1': {
                    '1': {'1': 'a', '2': 'b', '3': 'c'},
                    '2': {'1': 'd', '2': 'e', '3': 'f'},
                    '3': {'1': 'g', '2': 'h', '3': 'i'},
                },
                '2': {
                    '1': {'1': 'A', '2': 'B', '3': 'C'},
                    '2': {'1': 'D', '2': 'E', '3': 'F'},
                    '3': {'1': 'G', '2': 'H', '3': 'I'},
                },
                '3': {
                    '1': {'1': 'r', '2': 's', '3': 't'},
                    '2': {'1': 'u', '2': 'v', '3': 'w'},
                    '3': {'1': 'x', '2': 'y', '3': 'z'},
                },
                '4': {
                    '1': {'1': 'R', '2': 'S', '3': 'T'},
                    '2': {'1': 'U', '2': 'V', '3': 'W'},
                    '3': {'1': 'X', '2': 'Y', '3': 'Z'},
                },
            }
            return periodicity_map[str(periodicity)][str(size)][
                str(duration)
            ]

        def compute_timechar():
            """Function to compute the timechar"""
            # print(f'Compute timechar. Profileid: {profileid} T2: {T2}', 0, 5)
            if not isinstance(T2, bool):
                if T2 <= timedelta(seconds=5).total_seconds():
                    return '.'
                elif T2 <= timedelta(seconds=60).total_seconds():
                    return ','
                elif T2 <= timedelta(seconds=300).total_seconds():
                    return '+'
                elif T2 <= timedelta(seconds=3600).total_seconds():
                    return '*'
                else:
                    # Changed from 0 to ''
                    return ''
            else:
                return ''

        # Here begins the function's code
        try:
            # Update value of T2
            if now_ts and last_ts:
                T2 = now_ts - last_ts
            else:
                T2 = False
            # Are flows sorted?
            if T2 < 0:
                # Flows are not sorted!
                # What is going on here when the flows are not ordered?? Are we losing flows?
                # Put a warning
                print(
                    'Warning: Coming flows are not sorted -> Some time diff are less than zero.',
                    0,
                    2,
                )
                pass
        except TypeError:
            T2 = False
        # print("T2:{}".format(T2), 0, 1)
        # p = __database__.start_profiling()
        # Compute the rest
        periodicity, zeros = compute_periodicity(
            now_ts, last_ts, last_last_ts
        )
        duration = compute_duration()
        # print("Duration: {}".format(duration), 0, 1)
        size = compute_size()
        # print("Size: {}".format(size), 0, 1)
        letter = compute_letter()
        # print("Letter: {}".format(letter), 0, 1)
        timechar = compute_timechar()
        # print("TimeChar: {}".format(timechar), 0, 1)
#         print(
#             'Profileid: {}, Tuple: {}, Periodicity: {}, Duration: {}, Size: {}, Letter: {}. TimeChar: {}'.format(
#                 profileid,
#                 tupleid,
#                 periodicity,
#                 duration,
#                 size,
#                 letter,
#                 timechar,
#             ),
#             3,
#             0,
#         )
        # p = __database__.end_profiling(p)
        symbol = zeros + letter + timechar
        # Return the symbol, the current time of the flow and the T1 value
        return symbol, (last_ts, now_ts)
    except Exception as inst:
        # For some reason we can not use the output queue here.. check
        print('Error in compute_symbol in profilerProcess.', 0, 1)
        print('{}'.format(type(inst)), 0, 1)
        print('{}'.format(inst), 0, 1)
        print('{}'.format(traceback.format_exc()), 0, 1)


In [20]:
# hh[3402]

In [21]:
# column_values = hh[3402]

In [22]:
id_separator = __database__.getFieldSeparator()

In [23]:
id_separator

'_'

In [24]:
# column_values['uid'] = base64.b64encode(
#                     binascii.b2a_hex(os.urandom(9))
#                 ).decode('utf-8')
# uid = column_values['uid']
# flow_type = column_values['type']
# saddr = column_values['saddr']
# daddr = column_values['daddr']
# profileid = 'profile' + id_separator + str(saddr)

In [25]:
def get_timewindow(flowtime, profileid):
    """ "
    This function should get the id of the TW in the database where the flow belong.
    If the TW is not there, we create as many tw as necessary in the future or past until we get the correct TW for this flow.
    - We use this function to avoid retrieving all the data from the DB for the complete profile. We use a separate table for the TW per profile.
    -- Returns the time window id
    THIS IS NOT WORKING:
    - The empty profiles in the middle are not being created!!!
    - The Dtp ips are stored in the first time win
    """
    try:
        # First check if we are not in the last TW. Since this will be the majority of cases

        try:
            if not profileid:
                # profileid is None if we're dealing with a profile
                # outside of home_network when this param is given
                return False
            [
                (lasttwid, lasttw_start_time)
            ] = __database__.getLastTWforProfile(profileid)
            lasttw_start_time = float(lasttw_start_time)
            lasttw_end_time = lasttw_start_time + width
            flowtime = float(flowtime)
#             print(
#                 'The last TW id for profile {} was {}. Start:{}. End: {}'.format(
#                     profileid, lasttwid, lasttw_start_time, lasttw_end_time
#                 ),
#                 3,
#                 0,
#             )
            # There was a last TW, so check if the current flow belongs here.
            if (
                lasttw_end_time > flowtime
                and lasttw_start_time <= flowtime
            ):
#                 print(
#                     'The flow ({}) is on the last time window ({})'.format(
#                         flowtime, lasttw_end_time
#                     ),
#                     3,
#                     0,
#                 )
                twid = lasttwid
            elif lasttw_end_time <= flowtime:
                # The flow was not in the last TW, its NEWER than it
#                 print(
#                     'The flow ({}) is NOT on the last time window ({}). Its newer'.format(
#                         flowtime, lasttw_end_time
#                     ),
#                     3,
#                     0,
#                 )
                amount_of_new_tw = int(
                    (flowtime - lasttw_end_time) / width
                )
#                 print(
#                     'We have to create {} empty TWs in the midle.'.format(
#                         amount_of_new_tw
#                     ),
#                     3,
#                     0,
#                 )
                temp_end = lasttw_end_time
                for id in range(0, amount_of_new_tw + 1):
                    new_start = temp_end
                    twid = __database__.addNewTW(profileid, new_start)
#                     print(
#                         'Creating the TW id {}. Start: {}.'.format(
#                             twid, new_start
#                         ),
#                         3,
#                         0,
#                     )
                    temp_end = new_start + width
                # Now get the id of the last TW so we can return it
            elif lasttw_start_time > flowtime:
                # The flow was not in the last TW, its OLDER that it
#                 print(
#                     'The flow ({}) is NOT on the last time window ({}). Its older'.format(
#                         flowtime, lasttw_end_time
#                     ),
#                     3,
#                     0,
#                 )
                # Find out if we already have this TW in the past
                data = __database__.getTWofTime(profileid, flowtime)
                if data:
                    # We found a TW where this flow belongs to
                    (twid, tw_start_time) = data
                    return twid
                else:
                    # There was no TW that included the time of this flow, so create them in the past
                    # How many new TW we need in the past?
                    # amount_of_new_tw is the total amount of tw we should have under the new situation
                    amount_of_new_tw = int(
                        (lasttw_end_time - flowtime) / width
                    )
                    # amount_of_current_tw is the real amount of tw we have now
                    amount_of_current_tw = (
                        __database__.getamountTWsfromProfile(profileid)
                    )
                    # diff is the new ones we should add in the past. (Yes, we could have computed this differently)
                    diff = amount_of_new_tw - amount_of_current_tw
#                     print(
#                         'We need to create {} TW before the first'.format(
#                             diff + 1
#                         ),
#                         3,
#                         0,
#                     )
                    # Get the first TW
                    [
                        (firsttwid, firsttw_start_time)
                    ] = __database__.getFirstTWforProfile(profileid)
                    firsttw_start_time = float(firsttw_start_time)
                    # The start of the new older TW should be the first - the width
                    temp_start = firsttw_start_time - width
                    for id in range(0, diff + 1):
                        new_start = temp_start
                        # The method to add an older TW is the same as to add a new one, just the starttime changes
                        twid = __database__.addNewOlderTW(
                            profileid, new_start
                        )
#                         print(
#                             'Creating the new older TW id {}. Start: {}.'.format(
#                                 twid, new_start
#                             ),
#                             3,
#                             0,
#                         )
                        temp_start = new_start - width
        except ValueError:
            # There is no last tw. So create the first TW
            # If the option for only-one-tw was selected, we should create the TW at least 100 years before the flowtime, to cover for
            # 'flows in the past'. Which means we should cover for any flow that is coming later with time before the first flow
            if width == 9999999999:
                # Seconds in 1 year = 31536000
                startoftw = float(flowtime - (31536000 * 100))
            else:
                startoftw = float(flowtime)
            # Add this TW, of this profile, to the DB
            twid = __database__.addNewTW(profileid, startoftw)
            # print("First TW ({}) created for profile {}.".format(twid, profileid), 0, 1)
        return twid
    except Exception as e:
        print('Error in get_timewindow().', 0, 1)
        print('{}'.format(e), 0, 1)

In [26]:
# ts = column_values['starttime']
# if type(ts) == str:
#     datetime_obj = datetime.strptime(ts, timeformat)
#     starttime = datetime_obj.timestamp()
# else:
#     starttime = column_values['starttime'].timestamp()

In [27]:
# dur = column_values['dur']
# sport = column_values['sport']
# dport = column_values['dport']
# sport = column_values['sport']
# proto = column_values['proto']
# state = column_values['state']
# pkts = column_values['pkts']
# allbytes = column_values['bytes']
# spkts = column_values['spkts']
# sbytes = column_values['sbytes']
# endtime = column_values['endtime']
# appproto = column_values['appproto']
# direction = column_values['dir']
# dpkts = column_values['dpkts']
# dbytes = column_values['dbytes']
# smac = column_values.get('smac')
# dmac = column_values.get('dmac')

In [28]:
import ipaddress
# saddr = column_values['saddr']
# daddr = column_values['daddr']
# try:
#     saddr_as_obj = ipaddress.IPv4Address(saddr)
#     daddr_as_obj = ipaddress.IPv4Address(daddr)
#     # Is ipv4
# except ipaddress.AddressValueError:
#     # Is it ipv6?
#     try:
#         saddr_as_obj = ipaddress.IPv6Address(saddr)
#         daddr_as_obj = ipaddress.IPv6Address(daddr)
#     except ipaddress.AddressValueError:
#         # Its a mac
#         print(12)
# tupleid = f'{daddr_as_obj}-{dport}-{proto}'

In [29]:
import configparser
conf = configparser.ConfigParser(interpolation=None)
f = open("./StratosphereLinuxIPS/slips.conf", "r")
conf.read_file(f)

In [30]:
try:
    data = conf.get('parameters', 'time_window_width')
    width = float(data)
except ValueError:
    # Its not a float
    if 'only_one_tw' in data:
        # Only one tw. Width is 10 9s, wich is ~11,500 days, ~311 years
        width = 9999999999
except configparser.NoOptionError:
    # By default we use 3600 seconds, 1hs
    width = 3600

In [31]:
__database__.start(config=conf, redis_port=port)

In [32]:
from multiprocessing import Queue
outputqueue = Queue()
inputqueue = Queue()
__database__.setOutputQueue(outputqueue)

In [33]:
# twid = get_timewindow(starttime, profileid)

In [34]:
# twid

In [35]:
uid_list = []
symbols_list = []
llll = 0
for i in read_zeek_files("dataset/conn.log"):
    llll+=1
    print(llll, end='\r')
    column_values = i
    uid_list.append(i["data"]["uid"])
    column_values = process_line(column_values)
    column_values['uid'] = base64.b64encode(
                    binascii.b2a_hex(os.urandom(9))
                ).decode('utf-8')
    uid = column_values['uid']
    flow_type = column_values['type']
    saddr = column_values['saddr']
    daddr = column_values['daddr']
    profileid = 'profile' + id_separator + str(saddr)
    ts = column_values['starttime']
    if type(ts) == str:
        datetime_obj = datetime.strptime(ts, timeformat)
        starttime = datetime_obj.timestamp()
    else:
        starttime = column_values['starttime'].timestamp()
    dur = column_values['dur']
    sport = column_values['sport']
    dport = column_values['dport']
    sport = column_values['sport']
    proto = column_values['proto']
    state = column_values['state']
    pkts = column_values['pkts']
    allbytes = column_values['bytes']
    spkts = column_values['spkts']
    sbytes = column_values['sbytes']
    endtime = column_values['endtime']
    appproto = column_values['appproto']
    direction = column_values['dir']
    dpkts = column_values['dpkts']
    dbytes = column_values['dbytes']
    smac = column_values.get('smac')
    dmac = column_values.get('dmac')
    saddr = column_values['saddr']
    daddr = column_values['daddr']
    try:
        saddr_as_obj = ipaddress.IPv4Address(saddr)
        daddr_as_obj = ipaddress.IPv4Address(daddr)
        # Is ipv4
    except ipaddress.AddressValueError:
        # Is it ipv6?
        try:
            saddr_as_obj = ipaddress.IPv6Address(saddr)
            daddr_as_obj = ipaddress.IPv6Address(daddr)
        except ipaddress.AddressValueError:
            # Its a mac
            print(12)
    tupleid = f'{daddr_as_obj}-{dport}-{proto}'
    twid = get_timewindow(starttime, profileid)
#     print("profileid = ",profileid)
    symbol = compute_symbol(
                        profileid,
                        twid,
                        tupleid,
                        starttime,
                        dur,
                        allbytes,
                        tuple_key='OutTuples',
                    )
#     print(type(symbol))
    symbols_list.append(symbol)

In [51]:
# import pickle
# with open('uid.pickle', 'wb') as handle:
#     pickle.dump(uid_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
# import pickle
# with open('symbol.pickle', 'wb') as handle:
#     pickle.dump(symbols_list, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [33]:
# twid

In [36]:
len(uid_list) == len(set(uid_list))

True

In [37]:
len(symbols_list)

460887

In [38]:
aaa = []
for i in symbols_list:
    aaa.append(i[0])

In [39]:
len(aaa)

460887

In [40]:
# profileid

In [34]:
# hh[]

In [35]:
symbol = compute_symbol(
                        profileid,
                        twid,
                        tupleid,
                        starttime,
                        dur,
                        allbytes,
                        tuple_key='OutTuples',
                    )

Starting compute symbol. Profileid: profile_192.168.202.79, Tupleid 192.168.229.153-49160-tcp, time:1331901115.37 (<class 'float'>), dur:0.06000018119812012, size:322 3 0
Compute Periodicity: Profileid: profile_192.168.202.79, Tuple: 192.168.229.153-49160-tcp, T1=None, T2=None, TD=-1 3 0
Profileid: profile_192.168.202.79, Tuple: 192.168.229.153-49160-tcp, Periodicity: -1, Duration: 1, Size: 2, Letter: 4. TimeChar:  3 0


In [58]:
symbol

('4', (False, 1331901115.37))

In [42]:
set(aaa)

{'1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [41]:
aaa

['1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '4',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '4',
 '1',
 '1',
 '4',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '4',
 '1',
 '7',
 '4',
 '1',
 '1',
 '1',
 '4',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1'

In [3]:
# from StratosphereLinuxIPS.slips_files.core.profilerProcess import ProfilerProcess
# from StratosphereLinuxIPS.slips_files.core.inputProcess import InputProcess

# procs = ProfilerProcess(inputqueue, outputqueue,debug=True, config=conf,redis_port=6379, verbose=True)

------here----


In [4]:
# procs.start()

In [6]:
# procs.pid

137721

In [10]:
# profilerProcessQueue = Queue()

# input_information = "~/dataset/1/dns.log"
# input_type = 'stdin'
# line_type = 'zeek'
# redis_port=6379
# pcapfilter=None
# zeek_bro='zeek'

# inputProcess = InputProcess(
#                 outputqueue,
#                 profilerProcessQueue,
#                 input_type,
#                 input_information,
#                 conf,
#                 pcapfilter,
#                 zeek_bro,
#                 line_type,
#                 redis_port,
#             )

In [11]:
# inputProcess.start()

In [42]:
# __database__.getProfilesLen()

In [43]:
# `import redis

In [18]:
# port = 6379
# r = redis.StrictRedis(
#                 host='localhost',
#                 port=port,
#                 db=0,
#                 charset='utf-8',
#                 socket_keepalive=True,
#                 retry_on_timeout=True,
#                 decode_responses=True,
#                 health_check_interval=20,
#             )

In [ ]:
# id_separator = __database__.getFieldSeparator()
# profileid = 'profile' + id_separator + str(saddr)
# separator = '_'
# hash_id = profileid + separator + twid

In [ ]:
# r.hget()